In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [3]:
def transTime(split):
    split = split.split("/")
    split[1], split[0] = split[0], split[1]
    split = "/".join(split)
    return split

In [4]:
def getInterval(arrayLike):
    launched = transTime(arrayLike["launched"])
    deadline = transTime(arrayLike["deadline"])
    interval = pd.to_datetime(deadline)-pd.to_datetime(launched)
    return interval.days

In [5]:
def trans_category(arrayLike):
    cat_dic = []
    main_dic = []
    assert(arrayLike is not None)
    category = arrayLike["category"]
    main_category = arrayLike["main_category"]
    for ele in category:
        if ele not in cat_dic:
            cat_dic.append(ele)
    for ele in main_category:
        if ele not in main_dic:
            main_dic.append(ele)     
    return cat_dic, main_dic      

In [6]:
def trans_currency(arrayLike):
    curr_dic = []
    currency = arrayLike["currency"]
    for ele in currency:
        if ele not in curr_dic:
            curr_dic.append(ele)
    return curr_dic

In [7]:
def trans_country(arrayLike):
    country_dic = []
    country = arrayLike["country"]
    for ele in country:
        if ele not in country_dic:
            country_dic.append(ele)
    return country_dic

In [8]:
def label_generate(arrayLike):
    label = []
    country = arrayLike["state"]
    for ele in country:
        if ele not in label:
            label.append(ele)
    return label

In [9]:
missing_values = ["N,0\""]
dataset = pd.read_csv("kickstarter201801.csv",)
dataset = dataset.drop(["ID","name"],axis=1)
x = dataset["usd pledged"].median()
dataset["usd pledged"].fillna(x,inplace = True)
print(dataset["usd pledged"].isnull().any())
print(dataset)

False
               category main_category currency    deadline     goal  \
0                Poetry    Publishing      GBP  09/10/2015   1000.0   
1        Narrative Film  Film & Video      USD  01/11/2017  30000.0   
2        Narrative Film  Film & Video      USD  26/02/2013  45000.0   
3                 Music         Music      USD  16/04/2012   5000.0   
4          Film & Video  Film & Video      USD  29/08/2015  19500.0   
...                 ...           ...      ...         ...      ...   
378656      Documentary  Film & Video      USD  17/10/2014  50000.0   
378657   Narrative Film  Film & Video      USD  19/07/2011   1500.0   
378658   Narrative Film  Film & Video      USD  16/08/2010  15000.0   
378659       Technology    Technology      USD  13/02/2016  15000.0   
378660  Performance Art           Art      USD  16/08/2011   2000.0   

                launched  pledged     state  backers country  usd pledged  \
0       11/08/2015 12:12      0.0    failed        0      GB    

In [10]:
dataset["interval"] = dataset.apply(getInterval,axis=1)
dataset = dataset.drop(["deadline","launched"],axis=1)
cat_dic, main_dic = trans_category(dataset)
dataset["category"] = dataset["category"].apply(lambda x: cat_dic.index(x))
dataset["main_category"] = dataset["main_category"].apply(lambda x: main_dic.index(x))
curr_dic = trans_currency(dataset)
dataset["currency"] = dataset["currency"].apply(lambda x: curr_dic.index(x))
country_dic = trans_country(dataset)
dataset["country"] = dataset["country"].apply(lambda x: country_dic.index(x))
state_dic = label_generate(dataset)
y = dataset["state"].apply(lambda x: state_dic.index(x))
dataset = dataset.drop(["state"],axis=1)
print(dataset)

        category  main_category  currency     goal  pledged  backers  country  \
0              0              0         0   1000.0      0.0        0        0   
1              1              1         1  30000.0   2421.0       15        1   
2              1              1         1  45000.0    220.0        3        1   
3              2              2         1   5000.0      1.0        1        1   
4              3              1         1  19500.0   1283.0       14        1   
...          ...            ...       ...      ...      ...      ...      ...   
378656         8              1         1  50000.0     25.0        1        1   
378657         1              1         1   1500.0    155.0        5        1   
378658         1              1         1  15000.0     20.0        1        1   
378659        57             12         1  15000.0    200.0        6        1   
378660        75             10         1   2000.0    524.0       17        1   

        usd pledged  usd_pl

In [11]:
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.1, shuffle=True)

In [12]:
knn = KNeighborsClassifier()

print('training KNN...')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print('accuracy = ', accuracy)

training KNN...
accuracy =  0.8679325005941849


In [13]:
gn = GaussianNB()
print('training Gaussian...')
gn.fit(X_train, y_train)
y_pred = gn.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print('accuracy = ', accuracy)

training Gaussian...
accuracy =  0.5724773549528613


In [14]:
rf = RandomForestClassifier(random_state=1)
print('training random forest...')
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print('accuracy = ', accuracy)

training random forest...


In [ ]:
voting = VotingClassifier(estimators=[('knn',knn),('gaussian',gn),('random forest',rf)],voting='hard')
print('training voting classifier...')
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print('accuracy = ', accuracy)

training random forest...
accuracy =  0.8790239522539415
